In [1]:
from datasets import load_dataset
import pandas as pd
import os

base_path = "/content/drive/MyDrive/nlp_7th_sem/lab4"

train_file = f"{base_path}/train.csv"
validation_file = f"{base_path}/test.csv"

if not (os.path.exists(train_file) and os.path.exists(validation_file)):
    raise RuntimeError(f"Error: Dataset files not found. Please ensure {train_file} and {validation_file} are in the current directory.")

raw_datasets = load_dataset(
        "csv",
        data_files={
            "train": train_file,
            "validation": validation_file
        }
    )
print("Successfully loaded datasets:")
print(raw_datasets)


Successfully loaded datasets:
DatasetDict({
    train: Dataset({
        features: ['en', 'uk'],
        num_rows: 3500
    })
    validation: Dataset({
        features: ['en', 'uk'],
        num_rows: 1000
    })
})


In [2]:
from transformers import AutoTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer

MODEL_CHECKPOINT = "Helsinki-NLP/opus-mt-en-uk"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = MarianMTModel.from_pretrained(MODEL_CHECKPOINT)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
import numpy as np

def get_token_lengths(dataset, language_key):
    lengths = []

    if language_key == 'uk':
        with tokenizer.as_target_tokenizer():
            for text in dataset[language_key]:
                lengths.append(len(tokenizer.tokenize(text)))
    else:
        for text in dataset[language_key]:
            lengths.append(len(tokenizer.tokenize(text)))

    return lengths

train_en_lengths = get_token_lengths(raw_datasets['train'], 'en')
train_uk_lengths = get_token_lengths(raw_datasets['train'], 'uk')

print("--- English (Source) Token Lengths in Training Data ---")
print(f"Min Length: {np.min(train_en_lengths)}")
print(f"Max Length: {np.max(train_en_lengths)}")
print(f"Mean Length: {np.mean(train_en_lengths):.2f}")
print(f"95th Percentile Length: {np.percentile(train_en_lengths, 95)}")


print("\n--- Ukrainian (Target) Token Lengths in Training Data ---")
print(f"Min Length: {np.min(train_uk_lengths)}")
print(f"Max Length: {np.max(train_uk_lengths)}")
print(f"Mean Length: {np.mean(train_uk_lengths):.2f}")
print(f"95th Percentile Length: {np.percentile(train_uk_lengths, 95)}")

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


--- English (Source) Token Lengths in Training Data ---
Min Length: 4
Max Length: 43
Mean Length: 13.90
95th Percentile Length: 22.0

--- Ukrainian (Target) Token Lengths in Training Data ---
Min Length: 4
Max Length: 59
Mean Length: 15.20
95th Percentile Length: 27.0


In [13]:
max_input_length = 32
max_target_length = 36

def preprocess_function(examples):
    model_inputs = tokenizer(examples['en'], max_length=max_input_length, truncation=True, padding=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['uk'], max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00


In [10]:
%pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.5 MB/s eta 0:00:00


In [12]:
%pip install unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 42.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not curre

In [17]:
import evaluate
import numpy as np


metric_bleu = evaluate.load("sacrebleu")
metric_meteor = evaluate.load("meteor")
metric_comet = evaluate.load("comet")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    references_for_bleu_meteor = [[label] for label in decoded_labels]
    references_for_comet = decoded_labels

    bleu_result = metric_bleu.compute(predictions=decoded_preds, references=references_for_bleu_meteor)

    meteor_result = metric_meteor.compute(predictions=decoded_preds, references=references_for_bleu_meteor)

    sources = raw_datasets['validation']['en']
    comet_result = metric_comet.compute(
        predictions=decoded_preds,
        references=references_for_comet,
        sources=sources
    )

    return {
        "bleu": round(bleu_result["score"], 4),
        "meteor": round(meteor_result["meteor"], 4),
        "comet_score": round(comet_result["mean_score"], 4)
    }

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [20]:
from transformers import Seq2SeqTrainingArguments
from transformers import MarianMTModel, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir=f"{base_path}/opus_mt_en_uk_fine_tuned",
    eval_strategy="steps",
    eval_steps=200,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-717997596.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [21]:
print("--- Baseline Metrics BEFORE Fine-tuning ---")
baseline_metrics = trainer.evaluate()
print(baseline_metrics)

--- Baseline Metrics BEFORE Fine-tuning ---


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'eval_loss': 0.9664192795753479, 'eval_model_preparation_time': 0.0028, 'eval_bleu': 30.8987, 'eval_meteor': 0.5501, 'eval_comet_score': 0.8259, 'eval_runtime': 49.0335, 'eval_samples_per_second': 20.394, 'eval_steps_per_second': 1.285}


In [22]:
print("\n--- Starting Fine-tuning ---")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.



--- Starting Fine-tuning ---


Step,Training Loss,Validation Loss,Model Preparation Time,Bleu,Meteor,Comet Score
200,No log,0.404360,0.002800,42.542300,0.670200,0.858600
400,No log,0.378903,0.002800,44.439900,0.678800,0.863900
600,0.444200,0.370784,0.002800,44.599900,0.679400,0.863900


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/l

TrainOutput(global_step=657, training_loss=0.41379066943396536, metrics={'train_runtime': 242.7049, 'train_samples_per_second': 43.262, 'train_steps_per_second': 2.707, 'total_flos': 88983207936000.0, 'train_loss': 0.41379066943396536, 'epoch': 3.0})

In [23]:
print("\n--- Final Metrics AFTER Fine-tuning ---")
final_metrics = trainer.evaluate()
print(final_metrics)


--- Final Metrics AFTER Fine-tuning ---


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'eval_loss': 0.3704577386379242, 'eval_model_preparation_time': 0.0028, 'eval_bleu': 44.7703, 'eval_meteor': 0.6803, 'eval_comet_score': 0.8645, 'eval_runtime': 54.1249, 'eval_samples_per_second': 18.476, 'eval_steps_per_second': 1.164, 'epoch': 3.0}
